<a href="https://colab.research.google.com/github/ysooch0819/AI16-Projects/blob/main/Section2_project%EC%A4%80%EB%B9%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
!pip3 install riotwatcher

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59 kB 3.2 MB/s 


In [9]:
def getAPIkey():
    f = open('./riot_api.txt', 'r')
    return f.read()

In [17]:
import pandas as pd
import requests
import time
from urllib import parse
from tqdm import tqdm
import json
from pandas import json_normalize

In [26]:
# api_key = getAPIkey()
api_key = 'RGAPI-1e09a2bc-eb83-4179-ab08-7dc3877a554f'

In [41]:
def league_summoner(key, tier, country='kr',):
    request =requests.get(f'https://{country}.api.riotgames.com/tft/league/v1/{tier}?api_key={key}')
    return json.loads(request.content)

In [42]:
ch_summoner = league_summoner(api_key, tier='challenger')
ch_summoner

{'tier': 'CHALLENGER',
 'leagueId': '946274f2-16b6-376e-8d60-e07ef8f06a7f',
 'queue': 'RANKED_TFT',
 'name': "Orianna's Dragons",
 'entries': [{'summonerId': 'PMpwwtehZYUWHnAQsOQtX5mBaRLIj2pIKY7Ad3l6MbNRcu8',
   'summonerName': '포카칩퍼런거',
   'leaguePoints': 1173,
   'rank': 'I',
   'wins': 131,
   'losses': 98,
   'veteran': False,
   'inactive': False,
   'freshBlood': False,
   'hotStreak': False},
  {'summonerId': 'cmMqhUnmEojwMlyBfiS6yB1P8oWxj0glCKszU-GXevWr0_3U',
   'summonerName': '전설이 나만없어',
   'leaguePoints': 1001,
   'rank': 'I',
   'wins': 239,
   'losses': 191,
   'veteran': False,
   'inactive': False,
   'freshBlood': True,
   'hotStreak': False},
  {'summonerId': 'GAvUP1d0CZMDebC_MEBF2h43e-fOaSMICKprtI9FJvWuRdNHMemVRSWJYA',
   'summonerName': '영원한애정고양이',
   'leaguePoints': 1055,
   'rank': 'I',
   'wins': 244,
   'losses': 221,
   'veteran': False,
   'inactive': False,
   'freshBlood': False,
   'hotStreak': False},
  {'summonerId': 'lDLkTKucjmLbE8_U32F7pk6E9zUjA7_jBEsvAk

In [43]:
ch_summoner = json_normalize(ch_summoner['entries'])
ch_summoner

,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,PMpwwtehZYUWHnAQsOQtX5mBaRLIj2pIKY7Ad3l6MbNRcu8,포카칩퍼런거,1173,I,131,98,False,False,False,False
1,cmMqhUnmEojwMlyBfiS6yB1P8oWxj0glCKszU-GXevWr0_3U,전설이 나만없어,1001,I,239,191,False,False,True,False
2,GAvUP1d0CZMDebC_MEBF2h43e-fOaSMICKprtI9FJvWuRd...,영원한애정고양이,1055,I,244,221,False,False,False,False
3,lDLkTKucjmLbE8_U32F7pk6E9zUjA7_jBEsvAkQRWatE2QU,SanChess,1098,I,159,99,False,False,False,False
4,H2X8AtY2-1P_VSwK1d0CpU3anDnmufSjx3ggJLT-8Vb__w,윤태호와아이들,1080,I,197,177,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
295,VYxNXLN7vUx9cS41V7UIwVaPbhFVDs95ezatGVQetNlnpM...,game0jang,935,I,382,344,False,False,True,False
296,EEWYoMCLfqrivXShxBin-WPI5Ct8ycREgeW0wq_vgZFecrw,롤좀하는가,1089,I,216,169,False,False,True,False
297,jSwIJ9qsXW6M1fOTkLSh853ct1NU7YJFLqj7CMoMFr30CCE,Not Scythe,1009,I,277,229,False,False,False,False
298,n6JYBuwA-8aoLVPHX_X4DvmidCpogkMe9vGu693HgOFRyhw,아유하,1016,I,232,203,False,False,False,False


In [44]:
summonerId = ch_summoner['summonerId']
summonerId

0        PMpwwtehZYUWHnAQsOQtX5mBaRLIj2pIKY7Ad3l6MbNRcu8
1       cmMqhUnmEojwMlyBfiS6yB1P8oWxj0glCKszU-GXevWr0_3U
2      GAvUP1d0CZMDebC_MEBF2h43e-fOaSMICKprtI9FJvWuRd...
3        lDLkTKucjmLbE8_U32F7pk6E9zUjA7_jBEsvAkQRWatE2QU
4         H2X8AtY2-1P_VSwK1d0CpU3anDnmufSjx3ggJLT-8Vb__w
                             ...                        
295    VYxNXLN7vUx9cS41V7UIwVaPbhFVDs95ezatGVQetNlnpM...
296      EEWYoMCLfqrivXShxBin-WPI5Ct8ycREgeW0wq_vgZFecrw
297      jSwIJ9qsXW6M1fOTkLSh853ct1NU7YJFLqj7CMoMFr30CCE
298      n6JYBuwA-8aoLVPHX_X4DvmidCpogkMe9vGu693HgOFRyhw
299    SOBw1576zjDxdNDKYedzN99GHN69xwtP1JBwBqHpHfpWJc...
Name: summonerId, Length: 300, dtype: object

In [49]:
def summoner_info(summonerId , key, country='kr'):
    request =requests.get(f'https://{country}.api.riotgames.com/tft/summoner/v1/summoners/{summonerId}?api_key={key}')
    return json.loads(request.content)
from time import sleep

def summoner_puuid(summonerId):
    summ_puuid = pd.Series([])
    l = 100
    for i in summonerId:
        l += 1
        if l%100 == 0:
            sleep(120)
        si = summoner_info(i,api_key,'kr') 
        si = json_normalize(si)
        summ_puuid = summ_puuid.append(si['puuid'],ignore_index = True)
        
    return summ_puuid
puuid = summoner_puuid(summonerId)
puuid

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys


0      TGqXOq6Nt_KIWwos-ZnOp5umNs23E7T9IO4EGAFbwRyUiP...
1      oTkW9N8_j3yeytZd3OKtRjULL-yubULbMq7gOckzWjhKpj...
2      ArdzgbEoez4UOXtaI_Ij6VrumWUkGJYly0mAoXukCUIJBj...
3      b2-2LnrIb2JTcxvsDQzd4IdAwpnMszh-xnXByoawMJuBjz...
4      K_Gx0bEPwEPo8dTcKKnQt1hJiOOA9fIE92BLZSuWZMxaCH...
                             ...                        
295    jY71uy7e026NtuT2PeK0LCMwbbpjqZJU_HL3GZTtfmSWfT...
296    PPUnBU8uUjZZS74p22zLRH_3LVEpM4oXEArF1wvuCjQu3A...
297    u2tKE9unRx1klcxBq0T9s_nZ5zATmnbj9jc1F7MmVeWVrH...
298    xkBjqo8StYmA8zn9MwOsajLZn4zKv_4ZVC1qT5I47HZDgw...
299    FAifq1BCRidVAINbdGJsgEXPMNFmlG0UHC7aPCaLTxPXFz...
Length: 300, dtype: object

In [51]:
def get_matchid(puuid, key, n, region='asia'):
    matchid = []
    j = 100
    for i in puuid:
        j += 1
        if j&100 ==0:
            sleep(120)
        try:
            request = requests.get(f'http://{region}.api.riotgames.com/tft/match/v1/matches/by-puuid/{i}/ids?count={n}&api_key={key}')
            request = json.loads(request.content)
            matchid.extend(request)
        except:
            pass
    
    return list(set(matchid))


ch_matchid = get_matchid(puuid, api_key, 200)
ch_matchid

[]

In [52]:
# 나중에 다시 전부 가져오려면 힘드니 작업 디렉토리에 저장
import pickle
with open('ch_matchid_221123', 'wb') as f:
    pickle.dump(ch_matchid, f)

In [53]:
# 다시 불러오려면 요렇게
with open('ch_matchid_221123', 'rb') as f:
    list_variable = pickle.load(f)

In [ ]:
game_record = pd.DataFrame()

def match_info(matchid, key, region='asia'):
    l=100
    global game_record
    for i in matchid:
        l+=1
        if l%100 == 0:
            sleep(120)
        try:
            request = requests.get(f'https://{region}.api.riotgames.com/tft/match/v1/matches/{i}?api_key={key}')
            request = json.loads(request.content)
            request = json_normalize(request)
            game_record = pd.concat([game_record,request])
        except:
            pass
    return game_record

In [ ]:
temp_ch = match_info(ch_matchid, api_key)
temp_ch

In [ ]:
temp_ch.isnull().sum()

In [ ]:
# 반환받은 에러코드 종류
temp_ch['status.status_code'].unique()

In [ ]:
# 64개 정도의 에러코드는 무시하고, 에러코드가 빈 데이터들만 가져옴
ch_match = temp_ch[temp_ch['status.status_code'].isnull()]

In [ ]:
# 역시 받아오는데 오래 걸리기 때문에, csv형태로 재빨리 저장
ch_match.to_csv('ch_match_record_221123.csv')

In [ ]:
# 데이터 살펴보기
json_normalize(temp_ch['info.participants'])

In [ ]:
df = json_normalize(json_normalize(temp_ch['info.participants']).iloc[2])
df.columns

In [ ]:
# 'augments'는 증강체, 'traits'는 시너지이며 
# info-tft_game_type이 'pairs(더블업)'일 경우 들어있는 데이터가 조금 다를 수 있다.